# TP BE Météo

## Chargement des données

Lessgo

In [ ]:
library(sf)
library(ggplot2)
library("rnaturalearth")
library("rnaturalearthdata")
library(dplyr)

In [ ]:
# 31s à s'exécuter sur Stuart
data_full = "../data/donnees-synop-essentielles-omm.csv"
data_ext10 = "../data/donnees_extracted_10.csv"

full_data <- read.csv(data_ext10, 
            sep=",", 
            na.string=c("","null","NaN"),
            header=TRUE, 
            stringsAsFactors=TRUE) # SI IL Y A DES PROBLÈMES BIZARRES PLUS LOIN, METTRE À FALSE

In [ ]:
summary(full_data)
# RQE : le read_csv remplace les ' ' par des '.' dans les titres des colonnes

colnames(full_data)
# affiche les numéros de sttaions choisies
stations_in_file = unique(unlist(full_data["ID.OMM.station"]))
print(stations_in_file)

In [ ]:
# hardcoded directories
data_dir <- "../data/"
mesures_dir <-  paste(data_dir, "mesures/", sep="")
stations_csv <- paste(data_dir, "stations.csv", sep="")

stations_df <- read.csv(stations_csv, 
            sep=",", 
            na.string=c("","null","NaN"),
            header=TRUE, 
            stringsAsFactors=FALSE)

colnames(stations_df)

In [ ]:

my_sf <- st_as_sf(stations_df, coords = c('Longitude', 'Latitude'))

#Plot without world map:

ggplot(my_sf) + 
  geom_sf(aes(color = 'red'))


In [ ]:

worldmap <- ne_countries(scale = 'medium', type = 'map_units',
                         returnclass = 'sf')
class(worldmap)


ggplot() +
  geom_sf(data = worldmap, color = "black", fill = "#CAFFDA") +
  geom_point(data = stations_df, aes(x = Longitude, y = Latitude), color = "darkblue", size = 2) +
  coord_sf() +
  theme_minimal()

In [ ]:
ggplot() +
  geom_sf(data = worldmap[worldmap$name == 'France',], color = "black", fill = "#CAFFDA") +
  geom_point(data = stations_df[stations_df$ID.OMM.station , ], 
            aes(x = Longitude, y = Latitude), 
            color = "darkblue", size = 2) +
  coord_sf() +
  geom_text(data = stations_df[stations_df$Longitude > -10 & stations_df$Longitude < 10, ],
              aes(x = Longitude, y = Latitude, label = ID.OMM.station),
              size = 4, color = "black", nudge_y = 0.2) # Adjust nudge_y to avoid overlap  theme_minimal()

In [ ]:
histogramggplot <- function(xs, title, legx, legy ){
    ggplot(xs, aes(x=legx, y=legy, fill=legx)) +
    geom_boxplot() +
    theme_light() +
    scale_fill_brewer("Set3")
}

histogram <- function(xs, title, legx, legy){
    my_colors <- c("#FFA500", "#008000", "#1E90FF", "#FF1493")
    par(bg = 'white')
    boxplot(xs, xlab = legx, ylab = legy, las = 1, col = my_colors)
    grid()
}

chosen_stations = c("7020", "7015", "7027")
histogram(xs = full_data["Pression.au.niveau.mer"], title = "oui" ,legx = "non" ,legy = "ok")

In [ ]:

"
Return columns required, with only the rows related to chosen_stations.
"
ColumnSelector <- function(columns, chosen_stations){
    return( select(full_data %>% filter(ID.OMM.station %in% chosen_stations), all_of(columns)) )
}


select(full_data %>% filter(ID.OMM.station %in% chosen_stations), all_of(c("Pression.au.niveau.mer")))

In [ ]:

"
Return a df containing a column for each chosen_station
"
ColumnSplitSelector <- function(column, chosen_stations){
    # choose length
    maxsize = 0
    for(s in chosen_stations){
        l <- length(select(full_data %>% filter(ID.OMM.station == s), column)[[column]])
        if(l>maxsize){
            maxsize <- l
        }
    }
    print("done for maxsize")
    
    df <- data.frame(matrix(, nrow = maxsize))
    for(s in chosen_stations){
        df[s] <- select(full_data %>% filter(ID.OMM.station == s), column)[[column]]%>%slice(1:maxsize)
    }
    return(df)
}

head(ColumnSplitSelector("Pression.au.niveau.mer", chosen_stations))

[1] "\nReturn a df containing a column for each chosen_station\n"

[1] "done for maxsize"
[1] "7020"
      matrix...nrow...maxsize.
1                           NA
2                           NA
3                           NA
4                           NA
5                           NA
6                           NA
7                           NA
8                           NA
9                           NA
10                          NA
11                          NA
12                          NA
13                          NA
14                          NA
15                          NA
16                          NA
17                          NA
18                          NA
19                          NA
20                          NA
21                          NA
22                          NA
23                          NA
24                          NA
25                          NA
26                          NA
27                          NA
28                          NA
29                          NA
30                          NA
31   

ERROR: Error in UseMethod("slice"): no applicable method for 'slice' applied to an object of class "c('double', 'numeric')"
